In [ ]:
import glob
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
from sklearn.utils import shuffle
import tensorflow as tf 

In [ ]:
path = r'D:\Breast cancer material\MIAS_Preprocess\mias_preprocess_Part1\original'
folders=glob.glob(os.path.join(path,'*'))
label=[]
i=0
imgs=[]
for fol in folders:
    files=glob.glob(os.path.join(fol,'*'))
    for image in files:
        im=cv2.imread(image)
        im= cv2.resize(im,(160,160))
        imgs.append(im)
        label.append(i)
    i=i+1
    
imgs=np.array(imgs)
label=np.array(label)
imgs,label=shuffle(imgs,label)

In [ ]:
model = tf.keras.applications.EfficientNetB0(
    include_top=True,
    weights=None,
    input_tensor=None,
    input_shape=(160,160, 3),
    pooling=None,
    classes=3,
    classifier_activation="softmax"
)
model.summary()

In [ ]:
from tensorflow.keras import optimizers
optimizer=optimizers.SGD(lr=0.0001, momentum=0.9)
# Let's train the model using RMSprop
model.compile(loss='categorical_crossentropy',
              # optimizer=optimizers.RMSprop(lr=1e-4),
              optimizer=optimizer,
              metrics=['accuracy'])

In [ ]:
batch_size = 8
step_per_epoch = b.shape[0]/8

In [ ]:
import tensorflow.keras.callbacks as callbacks
model_path = 'effNet_B0.h5'
model_check_points = callbacks.ModelCheckpoint(filepath= model_path, monitor='val_loss', verbose=0, save_best_only=True,
                                                save_weights_only=False, mode='auto', save_freq='epoch',
                                                options=None)
ES = callbacks.EarlyStopping(
    monitor='val_loss', min_delta=0, patience=5, verbose=0,
    mode='auto', baseline=None, restore_best_weights=False )
monitor = 'categorical_accuracy'
# reduce_lr = callbacks.ReduceLROnPlateau(monitor=monitor, patience=3)


In [ ]:
tf.config.experimental_run_functions_eagerly(True)
tf.config.run_functions_eagerly(True)
history =  model.fit(imgs,b, epochs=200, batch_size = batch_size, validation_split=0.05, 
                    callbacks = [model_check_points])

In [ ]:
preds = model.predict(imgs)

In [ ]:
b = np.zeros((label.size, label.max()+1))
b[np.arange(label.size),label] = 1